<a href="https://colab.research.google.com/github/sahil1239/Fall-Prediction-ELC/blob/main/Model%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import random
import os
import pandas as pd
import re
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, losses
import pickle
from sklearn.metrics import classification_report, confusion_matrix

In [16]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [17]:
path = "./Early_Fall_Prediction"
with open(path+'/xc1_train.pkl','rb') as f:
  train_x = pickle.load(f)
with open(path+'/yc1_train.pkl','rb') as f:
  train_y = pickle.load(f)
with open(path+'/xc1_test.pkl','rb') as f:
  test_x = pickle.load(f)
with open(path+'/yc1_test.pkl','rb') as f:
  test_y = pickle.load(f)
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(669, 50, 9)
(669,)
(168, 50, 9)
(168,)


In [18]:
unique_values, counts = np.unique(train_y, return_counts=True)
print(unique_values, counts)

[0 1] [242 427]


In [19]:
unique_values, counts = np.unique(test_y, return_counts=True)
print(unique_values, counts)

[0 1] [ 59 109]


In [20]:
n_timesteps = 50
n_features = 9
n_outputs = 1

def hybrid_model():

    model = models.Sequential()

    # First CNN layer

    model.add(layers.Conv1D(kernel_size=5,
                            filters=64,
                            activation='relu',
                            input_shape=(n_timesteps, n_features)))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.MaxPooling1D(3, strides=2, padding="valid"))

    # Second CNN layer

    model.add(layers.Conv1D(kernel_size=5,
                            filters=64,
                            activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.MaxPooling1D(3, strides=2, padding="valid"))

    # Third CNN layer

    model.add(layers.Conv1D(kernel_size=5,
                            filters=64,
                            activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.MaxPooling1D(3, strides=2, padding="valid"))

    model.add(layers.Dense(64))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.5))

    # First LSTM
    model.add(layers.LSTM(64, return_sequences=True))
    model.add(layers.Dropout(0.5))

    # Second LSTM
    model.add(layers.LSTM(64, return_sequences=True))
    model.add(layers.Dropout(0.5))

    model.add(layers.Flatten())
    model.add(layers.Dense(n_outputs, activation='sigmoid'))


    print(model.summary())


    opt = optimizers.Adam(learning_rate=0.0005)
    lss = losses.BinaryFocalCrossentropy()
    model.compile(loss=lss, optimizer=opt, metrics=['accuracy'])

    return model

In [21]:
n_epochs = 200
model = hybrid_model()
history = model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs = n_epochs, batch_size = 64)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 46, 64)            2944      
                                                                 
 batch_normalization_4 (Bat  (None, 46, 64)            256       
 chNormalization)                                                
                                                                 
 re_lu_3 (ReLU)              (None, 46, 64)            0         
                                                                 
 max_pooling1d_3 (MaxPoolin  (None, 22, 64)            0         
 g1D)                                                            
                                                                 
 conv1d_4 (Conv1D)           (None, 18, 64)            20544     
                                                                 
 batch_normalization_5 (Bat  (None, 18, 64)           

In [22]:
y_pred =np.squeeze(model.predict(test_x))

6/6 [==============================] - 1s 4ms/step


In [23]:
for i in range(len(y_pred)):
  if y_pred[i]>0.5:
    y_pred[i]=1
  else:
    y_pred[i]=0

In [24]:
unique_values, counts = np.unique(test_y, return_counts=True)
print(unique_values, counts)

[0 1] [ 59 109]


In [25]:
unique_values, counts = np.unique(y_pred, return_counts=True)
print(unique_values, counts)

[0. 1.] [ 59 109]


In [26]:
print(classification_report(test_y, y_pred, target_names=['no fall', 'fall']))

              precision    recall  f1-score   support

     no fall       1.00      1.00      1.00        59
        fall       1.00      1.00      1.00       109

    accuracy                           1.00       168
   macro avg       1.00      1.00      1.00       168
weighted avg       1.00      1.00      1.00       168



In [27]:
print(confusion_matrix(test_y, y_pred))

[[ 59   0]
 [  0 109]]


In [ ]:
model.save("modelpresentation.h5")